In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
from autoeq.constants import PEQ_CONFIGS
from autoeq.batch_processing import batch_processing
ROOT_PATH = Path().resolve()
import ipywidgets as widgets
from IPython.display import display
from measurements.rtings.rtings_crawler import RtingsCrawler
from measurements.crinacle.crinacle_crawler import CrinacleCrawler
from measurements.oratory1990.oratory1990_crawler import Oratory1990Crawler
from measurements.average import average_measurements
from measurements.rename_measurements import rename_measurements
from results.prune_results import prune_results
from results.update_indexes import update_all_indexes
from webapp.create_data import write_entries_and_measurements

## Crawling and Parsing
Additional Python packages are required for processing the measurements:
```bash
python -m pip install -U -r measurements/dev-requirements.txt
```

Measurement crawlers require [Google Chrome](https://www.google.com/chrome/) installed and
[ChromeDriver](https://googlechromelabs.github.io/chrome-for-testing/) binary in the measurements folder (or anywhere
in the PATH).

Measurement crawlers also require C++. This should be installed by default on Linux but on Windows you need to install
Microsoft Visual Studio build tools for this. https://visualstudio.microsoft.com/downloads/ ->
"Tools for Visual Studio 2019" -> "Build Tools for Visual Studio 2019".

oratory1990 crawler requires Ghostscript installed: https://www.ghostscript.com/download/gsdnld.html

### Crinacle
Download measurement data from Drive folder to `measurements/crinacle/raw_data/` before running this!

* `IEM Measurements/IEC60318-4 IEM Measurements (TSV txt)` into `AutoEq/measurements/crinacle/raw_data/IEC60318-4 IEM Measurements (TSV txt)`
* `IEM Measurements/4620 IEM Measurements` into `AutoEq/measurements/crinacle/raw_data/4620 IEM Measurements`
* `HP Measurements/EARS + 711 (TSV txt) (Legacy)` into `AutoEq/measurements/crinacle/raw_data/EARS + 711 (TSV txt) (Legacy)`
* `GRAS 43AG-7` into `AutoEq/measurements/crinacle/raw_data/GRAS 43AG-7`

In [ ]:
crawler = CrinacleCrawler()
crawler.run()
display(crawler.widget)

In [ ]:
crawler.process()

### oratory1990
oratory1990 crawler fetches all measurements from https://www.reddit.com/r/oratory1990/wiki/index/list_of_presets/, downloads PDFs and reads the frequency response measurement data from the PDFs. Parsing the PDFs requires [Ghostscript](https://www.ghostscript.com/download/gsdnld.html) to be installed on the system.

In [4]:
crawler = Oratory1990Crawler()
crawler.run()
display(crawler.widget)

In [ ]:
crawler.process()

### Rtings
Rtings crawler fetches all measurements from https://www.rtings.com/headphones/1-[2,4,5]/graph and downloads raw FR JSON files and parses them.

In [ ]:
crawler = RtingsCrawler()
crawler.process()
display(crawler.widget)

## Rename Measurements
Sometimes measurements are named incorrectly or previously only one sample existed and now multiple samples have been measured and so the original one needs to be renamed as "<name> (sample 1)"

**TODO:** Use prompts for renaming

In [ ]:
renames = [
    {'old_name': '', 'new_name': '', 'dbs': ['']},
]
rename_measurements(renames, dry_run=False)

## Prune Results
Check if obsolete results (e.g. because of renaming) exist and remove them

In [10]:
prune_results(databases=['oratory1990'], dry_run=False)

Removed "oratory1990\in-ear\Campfire Audio IO"
Removed "oratory1990\in-ear\MEE Audio Pinnacle PX"
Removed "oratory1990\in-ear\MEE Audio Planamic"
Removed "oratory1990\in-ear\oBravo Cupid"
Removed "oratory1990\in-ear\SoftEars Cerberus"
Removed "oratory1990\in-ear\SoftEars RS10"
Removed "oratory1990\over-ear\Fostex T60RP"
Removed "oratory1990\over-ear\HIFIMAN Susvara"
Removed "oratory1990\over-ear\Koss ESP950"
Removed "oratory1990\over-ear\Shure SRH840 (sample 1)"


## Update Results
Creates new results from the measurements. `eq_kwargs` are parameters shared by all jobs.

In [5]:
eq_kwargs = {
    'parametric_eq': True, 'ten_band_eq': True, 'convolution_eq': True,
    'parametric_eq_config': [PEQ_CONFIGS['4_PEAKING_WITH_LOW_SHELF'], PEQ_CONFIGS['4_PEAKING_WITH_HIGH_SHELF']],
    'fs': [44100, 48000],
    'thread_count': 0,
}

#### oratory1990 Over-ear

In [5]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'oratory1990', 'data', 'over-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'oratory1990', 'over-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'harman_over-ear_2018_wo_bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/1 [00:00<?, ?it/s]

#### oratory1990 In-ear

In [98]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'oratory1990', 'data', 'in-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'oratory1990', 'in-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'autoeq_in-ear.csv'),
    bass_boost_gain=9.5, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/5 [00:00<?, ?it/s]

#### oratory1990 Earbud

In [6]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'oratory1990', 'data', 'earbud'),
    output_dir=ROOT_PATH.joinpath('results', 'oratory1990', 'earbud'),
    compensation=ROOT_PATH.joinpath('compensation', 'autoeq_in-ear.csv'),
    bass_boost_gain=0.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### crinacle GRAS 43AG-7 On-ear

In [100]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'crinacle', 'data', 'over-ear', 'GRAS 43AG-7'),
    output_dir=ROOT_PATH.joinpath('results', 'crinacle', 'GRAS 43AG-7 over-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'harman_over-ear_2018_wo_bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/49 [00:00<?, ?it/s]

#### crinacle EARS+711 Over-ear

In [101]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'crinacle', 'data', 'over-ear', 'EARS + 711'),
    output_dir=ROOT_PATH.joinpath('results', 'crinacle', 'EARS + 711 over-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'crinacle_harman_over-ear_2018_wo_bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/2 [00:00<?, ?it/s]

#### crinacle 4620 In-ear

In [102]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'crinacle', 'data', 'in-ear', 'Bruel & Kjaer 4620'),
    output_dir=ROOT_PATH.joinpath('results', 'crinacle', 'Bruel & Kjaer 4620 in-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'diffuse_field_5128_-1dBpoct.csv'),
    bass_boost_gain=0.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/8 [00:00<?, ?it/s]

#### crinacle 711 In-ear

In [103]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'crinacle', 'data', 'in-ear', '711'),
    output_dir=ROOT_PATH.joinpath('results', 'crinacle', '711 in-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'autoeq_in-ear.csv'),
    bass_boost_gain=9.5, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/109 [00:00<?, ?it/s]

#### Rtings Over-ear

In [9]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'rtings', 'data', 'over-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'Rtings', 'over-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'rtings_harman_over-ear_2018_wo_bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/2 [00:00<?, ?it/s]

#### Rtings In-ear

In [10]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'rtings', 'data', 'in-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'Rtings', 'in-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'rtings_autoeq_in-ear.csv'),
    bass_boost_gain=9.5, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### Rtings Earbud

In [11]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'rtings', 'data', 'earbud'),
    output_dir=ROOT_PATH.joinpath('results', 'Rtings', 'earbud'),
    compensation=ROOT_PATH.joinpath('compensation', 'rtings_autoeq_in-ear.csv'),
    bass_boost_gain=0.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### Innerfidelity In-ear

In [111]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'innerfidelity', 'data', 'over-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'Innerfidelity', 'over-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'innerfidelity_harman_over-ear_2018_wo_bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/14 [00:00<?, ?it/s]

#### Innerfidelity In-ear

In [112]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'innerfidelity', 'data', 'in-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'Innerfidelity', 'in-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'innerfidelity_autoeq_in-ear.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/10 [00:00<?, ?it/s]

#### Innerfidelity Earbud

In [113]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'innerfidelity', 'data', 'earbud'),
    output_dir=ROOT_PATH.joinpath('results', 'Innerfidelity', 'earbud'),
    compensation=ROOT_PATH.joinpath('compensation', 'innerfidelity_autoeq_in-ear.csv'),
    bass_boost_gain=0.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### Headphone.com Legacy Over-ear

In [114]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'headphonecom', 'data', 'over-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'Headphone.com Legacy', 'over-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'headphonecom_harman_over-ear_2018_wo_bass.csv'),
    bass_boost_gain=6.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

  0%|          | 0/12 [00:00<?, ?it/s]

#### Headphone.com Legacy In-ear

In [115]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'headphonecom', 'data', 'in-ear'),
    output_dir=ROOT_PATH.joinpath('results', 'Headphone.com Legacy', 'in-ear'),
    compensation=ROOT_PATH.joinpath('compensation', 'headphonecom_autoeq_in-ear.csv'),
    bass_boost_gain=9.5, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=True, **eq_kwargs)

0it [00:00, ?it/s]

#### Headphone.com Legacy Earbud

In [116]:
_ = batch_processing(
    input_dir=ROOT_PATH.joinpath('measurements', 'headphonecom', 'data', 'earbud'),
    output_dir=ROOT_PATH.joinpath('results', 'Headphone.com Legacy', 'earbud'),
    compensation=ROOT_PATH.joinpath('compensation', 'headphonecom_autoeq_in-ear.csv'),
    bass_boost_gain=0.0, bass_boost_fc=105, bass_boost_q=0.7,
    new_only=False, **eq_kwargs)

  0%|          | 0/10 [00:00<?, ?it/s]

## Update Indexes
Updates recommended results, full results, DB specific results, HeSuVi results and ranking table.

In [3]:
update_all_indexes()

Creating ranking index...


  0%|          | 0/3649 [00:00<?, ?it/s]

Creating recommendations index...
Creating full index...
Creating source indices...
Creating HeSuVi ZIP archive...


  0%|          | 0/3649 [00:00<?, ?it/s]

#### Data for webapp

In [ ]:
write_entries_and_measurements()

## Deploy
1. Add files to Git, commit and push
2. Upload webapp data to server

# Sandbox
Don't run these! Random exploration while developing.

In [1]:
%load_ext autoreload
%autoreload 2

In [53]:
from measurements.name_index import NameItem, NameIndex
from measurements.crinacle.crinacle_crawler import CrinacleCrawler
from measurements.oratory1990.oratory1990_crawler import Oratory1990Crawler
from measurements.rtings.rtings_crawler import RtingsCrawler
from pathlib import Path
from tqdm.auto import tqdm
import re
import requests
from selenium.webdriver.common.by import By
import json
from bs4 import BeautifulSoup

In [ ]:
crawler = CrinacleCrawler()

In [5]:
crawler = Oratory1990Crawler(redownload=False)

In [184]:
crawler = RtingsCrawler()
crawler.crawl()
crawler.create_prompts(max_prompts=20)
crawler.reload_ui()
display(crawler.widget)

  0%|          | 0/757 [00:00<?, ?it/s]

In [ ]:
crawler.process(new_only=True)

## Reworking Rtings Crawl

In [206]:
from seleniumwire import webdriver
from bs4 import BeautifulSoup
import json

In [132]:
driver = webdriver.Chrome()

In [153]:
driver.get('https://www.rtings.com/headphones/1-5/graph')

In [154]:
label = driver.find_element(By.XPATH, '//*[contains(text(), "Show untested products")]')

In [155]:
checkbox = label.find_element(By.XPATH, "./..")

In [156]:
checkbox.click()

In [186]:
dropdown = crawler.driver.find_element(By.CLASS_NAME, 'graph_tool_section-header')

In [187]:
dropdown.click()

In [188]:
options = dropdown.find_elements(By.CLASS_NAME, 'simple_dropdown-option')

In [190]:
options[6].get_attribute('outerHTML')

'<div class="simple_dropdown-option is-selected"><p style="color:inherit;">AKG K702</p><p></p></div>'

In [185]:
driver.requests[-1].date

datetime.datetime(2023, 10, 22, 15, 4, 45, 936437)

In [ ]:
for req in driver.requests:
    print(req)

In [66]:
html = crawler.driver.find_element(By.TAG_NAME, 'html').get_attribute('outerHTML')

In [69]:
'1more quad' in str(html).lower()

True

In [117]:
document = crawler.get_beautiful_soup('https://www.rtings.com/headphones/1-5/graph')

In [ ]:
dropdown = document.find(class_='graph_tool_section-header')
for el in dropdown.find_all(class_='simple_dropdown-option'):
    print(el.text)

In [ ]:
function(a) {
  if (a.target === a.currentTarget || a.timeStamp >= s || a.timeStamp <= 0 || a.target.ownerDocument !== document) return i.apply(this, arguments)
}

In [6]:
set({1,2,3}).intersection({4,5,3,7})

{3}

In [196]:
# graph_tool_page is-rendering

In [199]:
html = requests.get('https://www.rtings.com/headphones/1-5/graph').text

In [203]:
doc = BeautifulSoup(html, 'html.parser')

In [219]:
test_bench = json.loads(doc.find(class_='graph_tool_page').get('data-props'))['test_bench']

In [224]:
for test in test_bench['tests']:
    print(test['original_id'], test['name'], test['equivalent_test_ids'])

3992 Frequency Response ['3992', '7903', '21550']
569 Consistency L ['569', '1019', '1209', '2056', '3185', '4007', '7913', '21560']
570 Consistency R ['570', '1020', '1210', '2057', '3186', '4008', '7914', '21561']
4011 Raw FR L ['4011', '7917', '21564']
4012 Raw FR R ['4012', '7918', '21565']
4039 Bass ['4039', '7920', '21567']
4033 Mid ['4033', '7927', '21574']
4046 Treble ['4046', '7933', '21580']
4003 Peaks/Dips Graph ['4003', '7939', '21586']
1621 Group Delay ['1621', '2074', '3189', '4014', '7943', '21590']
1965 PRTF ['1965', '2069', '3196', '4021', '7950', '21598']
3341 Weighted Harmonic Distortion ['3341', '4029', '7966', '21614']
349 Noise Isolation ['349', '1054', '1244', '2090', '3209', '4053', '7981', '21630']
353 Leakage ['353', '1061', '1251', '2097', '3217', '4060', '7988', '21638']
379 Microphone Frequency Response ['379', '1065', '1255', '2102', '3227', '4070', '8001', '21651']
385 SpNR ['385', '1072', '1262', '2112', '3235', '4078', '8010', '21660']
8143 Phase Respon

In [213]:
test_products = test_bench['comparable_products']

In [215]:
for product in test_products:
    print({'source_name': product['fullname'], 'product_id': product['id'], 'original_test_id': product['review']['test_results'][0]['test']['original_id']})
    print([test_result['test']['original_id'] for test_result in product['review']['test_results']])
    break

{'source_name': 'Bose QuietComfort Headphones Wireless', 'product_id': '44666', 'original_test_id': '3992'}
['3992', '569', '570', '4011', '4012', '4039', '4033', '4046', '4003', '1621', '1965', '3341', '349', '353', '379', '385', '8143']


In [50]:
for test_id in {'529', '570', '438', '349', '569', '353', '437', '436', '329'}:
    res = requests.post('https://www.rtings.com/api/v2/safe/graph_tool__product_graph_data_url', data={
        'named_version': 'public',
        'product_id': '388',
        'test_original_id': '323',
    })
    print(f'{test_id}: {res.json()}')

529: {'data': {'product': {'review': {'test_results': []}}}}
570: {'data': {'product': {'review': {'test_results': []}}}}
438: {'data': {'product': {'review': {'test_results': []}}}}
436: {'data': {'product': {'review': {'test_results': []}}}}
349: {'data': {'product': {'review': {'test_results': []}}}}
353: {'data': {'product': {'review': {'test_results': []}}}}
437: {'data': {'product': {'review': {'test_results': []}}}}
569: {'data': {'product': {'review': {'test_results': []}}}}
329: {'data': {'product': {'review': {'test_results': []}}}}


In [ ]:
print(json.dumps(requests.get(f'https://i.rtings.com{url}').json(), ensure_ascii=False, indent=2))

### Exploration of Rtings Data URLs
Different versions use different IDs for tests

1-4 and 1-5:
* `Raw FR L` = `['4011', '7917', '21564']`
* `Raw FR R` = `['4012', '7918', '21565']`
* `graph-raw-frequency-response-l` = `1344`
* `graph-raw-frequency-response-r` = `1343`

1-2:
* `Raw FR L` = `['1344', '2060', '3182']`
* `Raw FR R` = `['1343', '2061', '3183']`
* `graph-bass` = `436`
* `graph-mid` = `437`
* `graph-treble` = `438`

In [86]:
version = '1-2'
html = requests.get(f'https://www.rtings.com/headphones/{version}/graph').text
document = BeautifulSoup(html, 'html.parser')

In [87]:
test_bench = json.loads(document.find(class_='graph_tool_page').get('data-props'))['test_bench']

In [88]:
list(test_bench.keys())

['name', 'id', 'comparable_products', 'tests']

In [89]:
known_test_ids = {test['id']: {'name': test['name'], 'equivalent': test['equivalent_test_ids']} for test in test_bench['tests']}
for key, val in known_test_ids.items():
    print(key, val)

2037 {'name': 'Bass', 'equivalent': ['436', '996', '1186', '2037', '3163']}
2044 {'name': 'Mid', 'equivalent': ['437', '1007', '1197', '2044', '3170']}
2050 {'name': 'Treble', 'equivalent': ['438', '1013', '1203', '2050', '3176']}
2056 {'name': 'Consistency L', 'equivalent': ['569', '1019', '1209', '2056', '3185', '4007', '7913', '21560']}
2057 {'name': 'Consistency R', 'equivalent': ['570', '1020', '1210', '2057', '3186', '4008', '7914', '21561']}
2060 {'name': 'Raw FR L', 'equivalent': ['1344', '2060', '3182']}
2061 {'name': 'Raw FR R', 'equivalent': ['1343', '2061', '3183']}
2069 {'name': 'PRTF', 'equivalent': ['1965', '2069', '3196', '4021', '7950', '21598']}
2074 {'name': 'Group Delay', 'equivalent': ['1621', '2074', '3189', '4014', '7943', '21590']}
2075 {'name': 'Phase Response', 'equivalent': ['529', '1030', '1220', '2075', '3190', '4015']}
2085 {'name': 'Distortion', 'equivalent': ['329', '1037', '1227', '2085']}
2090 {'name': 'Noise Isolation', 'equivalent': ['349', '1054', '

In [90]:
unknown_test_ids = {}
for product in test_bench['comparable_products']:
    test_ids = [test_result['test']['original_id'] for test_result in product['review']['test_results']]
    for test_id in test_ids:
        if test_id not in known_test_ids:
            unknown_test_ids[test_id] = {'name': product['fullname'], 'id': product['id']}
unknown_test_ids

{'329': {'name': 'AKG K391-NC', 'id': '234'},
 '349': {'name': 'AKG K391-NC', 'id': '234'},
 '353': {'name': 'AKG K391-NC', 'id': '234'},
 '436': {'name': 'AKG K391-NC', 'id': '234'},
 '437': {'name': 'AKG K391-NC', 'id': '234'},
 '438': {'name': 'AKG K391-NC', 'id': '234'},
 '529': {'name': 'AKG K391-NC', 'id': '234'},
 '569': {'name': 'AKG K391-NC', 'id': '234'},
 '570': {'name': 'AKG K391-NC', 'id': '234'}}

In [91]:
for test_id, product in unknown_test_ids.items():
    res = requests.post('https://www.rtings.com/api/v2/safe/graph_tool__product_graph_data_url', data={
        'named_version': 'public',
        'product_id': product['id'],
        'test_original_id': test_id,
    })
    print(f'{test_id}: {res.json()}')

329: {'data': {'product': {'review': {'test_results': [{'graph_data_url': '/assets/products/mcuBmpTu/graph-distortion.json'}]}}}}
349: {'data': {'product': {'review': {'test_results': [{'graph_data_url': '/assets/products/l2VvLSvF/graph-isolation.json'}]}}}}
353: {'data': {'product': {'review': {'test_results': [{'graph_data_url': '/assets/products/pPwMCFvC/graph-leakage.json'}]}}}}
436: {'data': {'product': {'review': {'test_results': [{'graph_data_url': '/assets/products/liYFzpsm/graph-bass.json'}]}}}}
437: {'data': {'product': {'review': {'test_results': [{'graph_data_url': '/assets/products/KhmOchbD/graph-mid.json'}]}}}}
438: {'data': {'product': {'review': {'test_results': [{'graph_data_url': '/assets/products/wzSsQqzQ/graph-treble.json'}]}}}}
529: {'data': {'product': {'review': {'test_results': [{'graph_data_url': '/assets/products/WsFGD2nr/graph-phase-response.json'}]}}}}
569: {'data': {'product': {'review': {'test_results': [{'graph_data_url': '/assets/products/LK19W26h/graph-